<a href="https://colab.research.google.com/github/pkant-0/Intelligent-Document-Processing-and-Query-System-/blob/main/Intelligent_document_processing_withQuery_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps:

    Document Processing:
        Processed PDF files, extracted text, and split it into logical sections
    Information Extraction and Tagging:
        Extracted the required information such as equipment name, domain, model, manufacturer using regular expressions/simple NLP method.
    Vector Representation:
        Used pre-trained embeddings from a model like SentenceTransformers to convert the extracted text into vectors.
    Vector Database:
        Store the vectorized information and the associated metadata and extracted information in a custom vector database.
    Query Processing:
        Query the vector database by converting the query to a vector and finding the nearest vectors (documents).
    Response Generation:
        Combine the retrieved information to generate a response using a language model like GPT-4.

In [ ]:
%%capture
# Install PyMuPDF for PDF extraction
!pip install PyMuPDF

# Install sentence-transformers for embedding generation
!pip install sentence-transformers

# Install openai for GPT-4 integration
!pip install openai

# Install SQLite if you want to use it as a vector database (Optional)
!pip install sqlite-utils


# Document procesing by PyMuPDF for PDF Extraction.

In [ ]:
import fitz  # PyMuPDF
from google.colab import files

# Uploading PDFs
uploaded_files = files.upload()

pdf_texts = {}
for filename, file in uploaded_files.items():
    with fitz.open(filename) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
        pdf_texts[filename] = text


Saving CleanBot_Robotic_Vacuum_Cleaner_FAQ.pdf to CleanBot_Robotic_Vacuum_Cleaner_FAQ (1).pdf
Saving CleanPro_Washing_Machine_FAQ.pdf to CleanPro_Washing_Machine_FAQ (1).pdf
Saving CompuTech_Laptop_FAQ.pdf to CompuTech_Laptop_FAQ (1).pdf
Saving CoolTech_Refrigerator_FAQ.pdf to CoolTech_Refrigerator_FAQ (1).pdf
Saving EcoControl_Smart_Home_Thermostat_FAQ.pdf to EcoControl_Smart_Home_Thermostat_FAQ (1).pdf
Saving FitTech_Fitness_Tracker_FAQ.pdf to FitTech_Fitness_Tracker_FAQ (1).pdf
Saving PhotoPro_Digital_Camera_FAQ.pdf to PhotoPro_Digital_Camera_FAQ (1).pdf
Saving SoundWave_Wireless_Earbuds_FAQ.pdf to SoundWave_Wireless_Earbuds_FAQ (1).pdf
Saving TechMobile_Smartphone_FAQ.pdf to TechMobile_Smartphone_FAQ (1).pdf
Saving ViewMax_Smart_TV_FAQ.pdf to ViewMax_Smart_TV_FAQ (1).pdf


Dealing with multiple pdf file is critical specially in analyzing technical documents or reports, for this we need to extract, orgnize and search the content correctly.

However extracting keywords and structuring data is cruicial for retrival and efficient indexing for larger data.

In this notebook i am applyting with regular expression to extract structured information - from the task i know that i have to extract these information.

in other case i can try with:
1. document uploading
2. text extraction - regular expression (regex). However if the structure varies i can apply machine learning or natural language processing (NLP)
3. keyword extraction- based on objective and concepual understanding we can apply keywords extaction.

For this we have methods like:
a. rule based
b. Automated/NLP - based.

4. Inorder to search we need to apply indexing for search, we can use vector-based search engines like Weaviate or elasticsearch.



# Splitting Document into leagal sections

In [ ]:
# Spliting text into paragraphs
def split_into_sections(text):
    return text.split("\n\n")  # Split by double newlines (paragraphs)

# Applying this to all PDFs
pdf_sections = {}
for filename, text in pdf_texts.items():
    pdf_sections[filename] = split_into_sections(text)

# Now each PDF has sections of text (paragraphs)

# Information Extraction and Tagging
using regular expression to extract specific/custom information.

In [ ]:
import re

# Define patterns for extraction
equipment_pattern = r"(Equipment Name:\s*[\w\s]+)"
domain_pattern = r"(Domain:\s*[\w\s]+)"
model_pattern = r"(Model Number:\s*[\w\s]+)"
manufacturer_pattern = r"(Manufacturer:\s*[\w\s]+)"

def extract_info(section):
    equipment = re.findall(equipment_pattern, section)
    domain = re.findall(domain_pattern, section)
    model = re.findall(model_pattern, section)
    manufacturer = re.findall(manufacturer_pattern, section)

    return {
        "equipment": equipment[0] if equipment else "",
        "domain": domain[0] if domain else "",
        "model": model[0] if model else "",
        "manufacturer": manufacturer[0] if manufacturer else ""
    }

# Apply extraction to all sections
extracted_info = {}
for filename, sections in pdf_sections.items():
    extracted_info[filename] = [extract_info(section) for section in sections]

# Extracted information per section of each PDF


# Vector representation:

In [ ]:
from sentence_transformers import SentenceTransformer

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_vector_representation(text):
    return model.encode(text).tolist()

# Get vector for each section of text
section_vectors = {}
for filename, sections in pdf_sections.items():
    section_vectors[filename] = [get_vector_representation(section) for section in sections]


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Vector representation

Using SentenceTransformers to generate vector embeddings for each section.

In [ ]:
from sentence_transformers import SentenceTransformer

# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_vector_representation(text):
    return model.encode(text).tolist()

# Get vector for each section of text
section_vectors = {}
for filename, sections in pdf_sections.items():
    section_vectors[filename] = [get_vector_representation(section) for section in sections]


# Storing Vectors and metadata in custom vector database.

This time i am avoiding weaviate and storing vectors simply in-memory structure using dictionary.

In [ ]:
# Simple vector database (in-memory structure)
vector_db = []

# Store vectors with metadata
for filename, sections in pdf_sections.items():
    for i, section in enumerate(sections):
        vector_db.append({
            "filename": filename,
            "section": section,
            "info": extracted_info[filename][i],  # Store extracted info
            "vector": section_vectors[filename][i]  # Store vector
        })


# Query Processing

Inorder to process query, i am converting query into vector.
finding the nearest matching sections using cosine similarity.


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to find the closest match
def find_nearest_vectors(query_vector, top_k=3):
    vectors = np.array([entry['vector'] for entry in vector_db])
    similarities = cosine_similarity([query_vector], vectors)[0]  # Cosine similarity
    top_indices = similarities.argsort()[-top_k:][::-1]  # Top K most similar vectors
    return [vector_db[i] for i in top_indices]

# Query vector
query = "Smart vacuum cleaner"
query_vector = get_vector_representation(query)

# Find the nearest sections
nearest_sections = find_nearest_vectors(query_vector)

# Response Generation

after retriving nearest section we can now combine the relevent information and generate response using CPT-4.


It is noted that API-Key is removed.

However i tested with my key and its working.

In [ ]:
import openai

# Assuming API key is set
openai.api_key = "YoUr-API"

def generate_response(query, relevant_info):
    prompt = f"Answer the following query based on this information: {relevant_info}\nQuery: {query}"
    response = openai.Completion.create(
        model="gpt-4",
        prompt=prompt,
        max_tokens=150
    )
    return response.choices[0].text.strip()

# Combine relevant info for GPT-4
relevant_info = "\n".join([f"Equipment: {section['info']['equipment']}, Domain: {section['info']['domain']}, Model: {section['info']['model']}, Manufacturer: {section['info']['manufacturer']}" for section in nearest_sections])

# Generate response
response = generate_response(query, relevant_info)
print(f"Response: {response}")


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
%%capture
!pip install --upgrade openai
!openai migrate
!pip install openai==0.28

In [ ]:
import openai

# Assuming API key is set
openai.api_key = "Tried_MyAPI"
def generate_response(query, relevant_info):
    prompt = f"Answer the following query based on this information: {relevant_info}\nQuery: {query}"

    # Updated API call for v1.0.0+
    response = openai.completions.create(
        model="gpt-4",
        prompt=prompt,
        max_tokens=150
    )
    # Extract the response text from the new structure
    return response.choices[0].text.strip()

# Example usage
nearest_sections = [
    {"info": {"equipment": "Generator", "domain": "Energy", "model": "X200", "manufacturer": "GenCorp"}},
    {"info": {"equipment": "Solar Panel", "domain": "Energy", "model": "SP100", "manufacturer": "SunPower"}}
]

query = "How should I maintain my solar panel?"
relevant_info = "\n".join([f"Equipment: {section['info']['equipment']}, Domain: {section['info']['domain']}, Model: {section['info']['model']}, Manufacturer: {section['info']['manufacturer']}" for section in nearest_sections])

# Generate response
response = generate_response(query, relevant_info)
print(f"Response: {response}")
